In [1]:
import os
from utils import *
import keras.layers.convolutional as convolutional
from keras.models import Sequential
import json
%matplotlib inline
from __future__ import print_function, division
import numpy
from os import listdir
import pandas as pd
from shutil import copyfile, rmtree
import allnew_vgg

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


The best approach would be to create a bunch of models, each time holding out one driver, and then average the validation across all of them (you could also average the predictions across all of them, like the ensembling we did in MNIST last week) But I wouldn't bother with that until you had done all the experimenting you wanted to do, since it adds a lot of time to each experiment.

In [15]:
def mn_valid(validation_frac=.3, local=False):
    if local:
        path = '/Users/matthew.negrin/deeplearning/courses/deeplearning1/nbs/data/statefarm/'
    else:
        path = '/home/ubuntu/matt/courses/deeplearning1/nbs/data/statefarm/'
    direct_list = ['valid/'+x[0].split('/')[-1] for x in os.walk(path+'train/') if x[0].split('/')[-1] != '']
    direct_list.insert(0, "valid")
    for directory in direct_list:
        if not os.path.exists(path+directory):
            os.makedirs(path+directory)
    dil = pd.read_csv(path + 'driver_imgs_list.csv')
    valid_subjects = dil.groupby(['subject']).size().sample(frac=validation_frac).keys().tolist()
    valid_frame = dil[dil['subject'].isin(valid_subjects)]
    for (subject, classname, img) in valid_frame.values:
        source = '{}train/{}/{}'.format(path, classname, img)
        target = source.replace('train', 'valid')
        print('mv {} {}'.format(source, target))
        os.rename(source, target)

def rm_sample_set():
    path = '/home/ubuntu/matt/courses/deeplearning1/nbs/data/statefarm/'
    rmtree(path+'sample/')
        
def mn_sample():
    path = '/home/ubuntu/matt/courses/deeplearning1/nbs/data/statefarm/'
    os.makedirs(path+'sample')
    os.makedirs(path+'sample/train')
    os.makedirs(path+'sample/valid')
    os.makedirs(path+'sample/models')
    os.makedirs(path+'sample/results')
    direct_list = [x[0].split('/')[-1] for x in os.walk(path+'train/') if x[0].split('/')[-1] != '']
    for d in direct_list:
        os.makedirs(path+'sample/train/'+d)
        os.mkdir(path+'sample/valid/'+d)
    tg = glob(path+'train/c?/*.jpg')
    shuf = np.random.permutation(tg)
    for i in range(1500): copyfile(shuf[i],path+'sample/train/' + shuf[i].split('/')[-2]+'/'+shuf[i].split('/')[-1])
    vg = glob(path+'valid/c?/*.jpg')
    vshuf = np.random.permutation(vg)
    for i in range(1000): copyfile(vshuf[i],path+'sample/valid/' + vshuf[i].split('/')[-2]+'/'+vshuf[i].split('/')[-1])    

In [3]:
# mn_valid()
# mn_sample()
# rm_sample_set()

In [12]:
current_dir = os.getcwd()
# data_dir = current_dir + '/data/statefarm/' #sample/
data_dir = current_dir + '/data/statefarm/sample/'
results_path=data_dir + 'results/'
train_path=data_dir + 'train/'
valid_path=data_dir + 'valid/'
test_path=data_dir + 'test/'
model_path=data_dir + 'models/'

In [6]:
batch_size=8

## Create Batches

In [142]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

def conv2(batches):
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [7]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(train_path, gen_t, batch_size=batch_size)
val_batches = get_batches(valid_path, batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [10]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(data_dir)

Found 15671 images belonging to 10 classes.
Found 6753 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


In [13]:
trn = get_data(train_path)
val = get_data(valid_path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [16]:
save_array(data_dir+'results/val.dat', val)
save_array(data_dir+'results/trn.dat', trn)
# val = load_array(path+'results/val.dat')
# trn = load_array(path+'results/trn.dat')

In [17]:
type(trn)

numpy.ndarray

In [146]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 33s - loss: 2.2838 - acc: 0.2693 - val_loss: 2.1581 - val_acc: 0.3030
Epoch 2/2
1500/1500 [==============================] - 33s - loss: 1.7052 - acc: 0.4447 - val_loss: 1.6523 - val_acc: 0.4790
Epoch 1/4
1500/1500 [==============================] - 34s - loss: 1.5338 - acc: 0.4887 - val_loss: 1.6045 - val_acc: 0.4600
Epoch 2/4
1500/1500 [==============================] - 32s - loss: 1.4162 - acc: 0.5093 - val_loss: 1.4913 - val_acc: 0.5350
Epoch 3/4
1500/1500 [==============================] - 33s - loss: 1.3188 - acc: 0.5720 - val_loss: 1.4043 - val_acc: 0.5080
Epoch 4/4
1500/1500 [==============================] - 33s - loss: 1.2640 - acc: 0.5867 - val_loss: 1.2721 - val_acc: 0.5950


In [147]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
1500/1500 [==============================] - 33s - loss: 1.1632 - acc: 0.6200 - val_loss: 1.4224 - val_acc: 0.5550
Epoch 2/5
1500/1500 [==============================] - 32s - loss: 1.0810 - acc: 0.6553 - val_loss: 1.3246 - val_acc: 0.5780
Epoch 3/5
1500/1500 [==============================] - 33s - loss: 1.0340 - acc: 0.6740 - val_loss: 1.2650 - val_acc: 0.6030
Epoch 4/5
1500/1500 [==============================] - 32s - loss: 1.0163 - acc: 0.6667 - val_loss: 1.0990 - val_acc: 0.6510
Epoch 5/5
1500/1500 [==============================] - 32s - loss: 1.0364 - acc: 0.6600 - val_loss: 1.4614 - val_acc: 0.5510


In [148]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/25
1500/1500 [==============================] - 34s - loss: 0.9446 - acc: 0.7000 - val_loss: 1.0697 - val_acc: 0.6660
Epoch 2/25
1500/1500 [==============================] - 32s - loss: 0.9120 - acc: 0.7287 - val_loss: 1.1718 - val_acc: 0.6330
Epoch 3/25
1500/1500 [==============================] - 33s - loss: 0.9165 - acc: 0.7113 - val_loss: 1.1122 - val_acc: 0.6380
Epoch 4/25
1500/1500 [==============================] - 32s - loss: 0.8506 - acc: 0.7373 - val_loss: 1.1121 - val_acc: 0.6100
Epoch 5/25
1500/1500 [==============================] - 32s - loss: 0.7845 - acc: 0.7553 - val_loss: 0.9349 - val_acc: 0.6820
Epoch 6/25
1500/1500 [==============================] - 33s - loss: 0.7507 - acc: 0.7640 - val_loss: 1.0202 - val_acc: 0.6560
Epoch 7/25
1500/1500 [==============================] - 33s - loss: 0.7962 - acc: 0.7540 - val_loss: 0.9893 - val_acc: 0.6760
Epoch 8/25
1500/1500 [==============================] - 33s - loss: 0.7403 - acc: 0.7647 - val_loss: 0.9372 - val_acc:

KeyboardInterrupt: 